In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, NonNegativeInt
from typing import List
from random import sample 

First, let's create a loader and load reviews from tv-reviews.csv into memory

In [3]:
# TODO: load reviews from tv-reviews.csv
from langchain.document_loaders.csv_loader import CSVLoader
data = CSVLoader("./data/tv-reviews.csv").load()

Then, let's initialize our LLM

In [4]:
model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = OpenAI(model_name=model_name, temperature=temperature, max_tokens=500)

/Users/diegofernandezgil/projects/personal-page/.venv/lib/python3.11/site-packages/langchain_community/llms/openai.py:249: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/Users/diegofernandezgil/projects/personal-page/.venv/lib/python3.11/site-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


Now, let's setup our parser and a template

In [13]:
class ReviewSentiment(BaseModel):
    positives: List[NonNegativeInt] = Field(
        description="index of a positive TV review, starting from 0"
    )
    negatives: List[NonNegativeInt] = Field(
        description="index of a negative TV review, starting from 0"
    )


parser = PydanticOutputParser(pydantic_object=ReviewSentiment)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"positives": {"title": "Positives", "description": "index of a positive TV review, starting from 0", "type": "array", "items": {"type": "integer", "minimum": 0}}, "negatives": {"title": "Negatives", "description": "index of a negative TV review, starting from 0", "type": "array", "items": {"type": "integer", "minimum": 0}}}, "required": ["positives", "negatives"]}
```


In [25]:
# TODO: setup a template with partial and input variables
prompt = PromptTemplate(
    template="{question}\n{format_instructions}\nContext: {context}",
    input_variables=["question", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

Pick 3 sample reviews to classify - LLMs have a limited context window they can work with. In later exercises, we'll see how to deal with that differently

In [26]:
sample(data, k=3)

[Document(page_content="TV Name: Imagix Pro\nReview Title: Unmatched Clarity\nReview Rating: 10\nReview Text: I cannot express enough how impressed I am with the clarity of the Imagix Pro. Every detail is so sharp and lifelike, it's like I can reach out and touch the images on the screen. The colors are vibrant and realistic, making everything look stunning. It truly enhances my movie-watching experience!", metadata={'source': './data/tv-reviews.csv', 'row': 2}),
 Document(page_content='TV Name: Imagix Pro\nReview Title: Exceptional Customer Service\nReview Rating: 10\nReview Text: I had a minor issue with my Imagix Pro, but the customer service team was exceptional. They were prompt in addressing my concern and guided me through the troubleshooting process. They even offered a quick replacement when the issue persisted. Kudos to their excellent support!', metadata={'source': './data/tv-reviews.csv', 'row': 9}),
 Document(page_content="TV Name: Imagix Pro\nReview Title: Unreliable Conn

In [27]:
# TODO: pick 3 random reviews and save them into reviews_to_classify variable
reviews_to_classify = sample(data, k=3)

# generate textual prompt from the prompt template
question = """
    Review TVs provided in the context. 
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs. 
    If there are no positive or negative reviews, output an empty JSON array. 
"""
query = prompt.format(context = context, question = question)

In [28]:
question = """
    Review TVs provided in the context. 
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs. 
    If there are no positive or negative reviews, output an empty JSON array. 
"""
context = "\n".join(review.page_content for review in reviews_to_classify)

query = prompt.format(context=context, question=question)
print(query)


    Review TVs provided in the context. 
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs. 
    If there are no positive or negative reviews, output an empty JSON array. 

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"positives": {"title": "Positives", "description": "index of a positive TV review, starting from 0", "type": "array", "items": {"type": "integer", "minimum": 0}}, "negatives": {"title": "Negatives", "description": "index of a negative TV review, starting from 0", "type": "a

Finally, let's send our query to LLM and use the parser we setup to parse an output into a Python object

In [29]:
output = llm(query)
print(output)


{
    "positives": [1, 2],
    "negatives": [0]
}


In [30]:
result = parser.parse(output)
result

ReviewSentiment(positives=[1, 2], negatives=[0])

In [31]:
# TODO: query LLM, then parse output into the result variable
print("Positives:\n" + "\n".join([reviews_to_classify[i].page_content for i in result.positives]))

Positives:
TV Name: Imagix Pro
Review Title: Outstanding Value for Money
Review Rating: 9
Review Text: The Imagix Pro is a fantastic value for money. Considering its high-quality performance, impressive features, and sleek design, it offers more bang for the buck compared to other TVs in the market. I am extremely satisfied with my purchase.
TV Name: Imagix Pro
Review Title: Impressive Features
Review Rating: 8
Review Text: The Imagix Pro is packed with impressive features that enhance my viewing experience. The smart functionality allows me to easily stream my favorite shows and movies. The remote control is user-friendly and has convenient shortcuts. The slim design is sleek and fits perfectly in my living room. The only downside is that the sound could be better, but overall, I'm satisfied.


In [32]:
print(
    "Negatives:\n"
    + "\n".join([reviews_to_classify[i].page_content for i in result.negatives])
)

Negatives:
TV Name: Imagix Pro
Review Title: Unreliable Connectivity
Review Rating: 4
Review Text: One major issue I have encountered with the Imagix Pro is its unreliable connectivity. The Wi-Fi frequently disconnects, causing buffering issues during streaming. The TV also struggles to maintain a stable Bluetooth connection. It's frustrating, especially considering the price I paid.
